## Muralidhar U64546777
## Assignment 04 logi,svm,dtree

## 1 import required packages

In [1]:
import pandas as pd 
import numpy as np 

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from sklearn.impute import SimpleImputer

from sklearn.model_selection import GridSearchCV,RandomizedSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import confusion_matrix


import warnings
warnings.filterwarnings("ignore")
np.random.seed(568)

## 2 load & clean data

In [2]:
df=pd.read_csv(r'~/Murali/DSP/Week 4/UniversalBank.csv')
df.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  5000 non-null   int64  
 1   Age                 5000 non-null   int64  
 2   Experience          5000 non-null   int64  
 3   Income              5000 non-null   int64  
 4   ZIP Code            5000 non-null   int64  
 5   Family              5000 non-null   int64  
 6   CCAvg               5000 non-null   float64
 7   Education           5000 non-null   int64  
 8   Mortgage            5000 non-null   int64  
 9   Personal Loan       5000 non-null   int64  
 10  Securities Account  5000 non-null   int64  
 11  CD Account          5000 non-null   int64  
 12  Online              5000 non-null   int64  
 13  CreditCard          5000 non-null   int64  
dtypes: float64(1), int64(13)
memory usage: 547.0 KB


In [4]:
df.describe()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000
mean,2500.500000,45.338400,20.104600,73.774200,93152.503000,2.396400,1.937938,1.881000,56.498800,0.096000,0.104400,0.06040,0.596800,0.294000
std,1443.520003,11.463166,11.467954,46.033729,2121.852197,1.147663,1.747659,0.839869,101.713802,0.294621,0.305809,0.23825,0.490589,0.455637
min,1.000000,23.000000,-3.000000,8.000000,9307.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
25%,1250.750000,35.000000,10.000000,39.000000,91911.000000,1.000000,0.700000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
50%,2500.500000,45.000000,20.000000,64.000000,93437.000000,2.000000,1.500000,2.000000,0.000000,0.000000,0.000000,0.00000,1.000000,0.000000
75%,3750.250000,55.000000,30.000000,98.000000,94608.000000,3.000000,2.500000,3.000000,101.000000,0.000000,0.000000,0.00000,1.000000,1.000000
max,5000.000000,67.000000,43.000000,224.000000,96651.000000,4.000000,10.000000,3.000000,635.000000,1.000000,1.000000,1.00000,1.000000,1.000000


df.isna().sum()

In [5]:
df.CreditCard.value_counts()

0    3530
1    1470
Name: CreditCard, dtype: int64

In [6]:
df['Securities Account'].value_counts()

0    4478
1     522
Name: Securities Account, dtype: int64

## remove unwanted data col

In [7]:
df =df.drop(columns=['ID', 'ZIP Code'])

## 3 split the data & transform

In [8]:
Universal_Col=df.columns
type(Universal_Col)

pandas.core.indexes.base.Index

In [9]:
predictors=Universal_Col.drop('Securities Account')
target='Securities Account'

In [10]:
X_train,X_test, y_train, y_test=train_test_split(df[predictors], df[target], test_size=0.30, random_state=46)

In [11]:
SS = StandardScaler()
X_train = SS.fit_transform(X_train)
X_test = SS.transform(X_test)

## 4 train the model with above data

### a. logistic regression grid & random Search

In [12]:
%%time
# parameters for grid & random search Cv for Logistic
score_measure = "precision"
kfolds = 5

param = {
    'solver'  : ['liblinear','newton-cg', 'lbfgs'],
    'penalty' : ['l1','l2'], 
    'C'       : np.logspace(-2,5,6)
}

# call Logistic regression 
LR=LogisticRegression()

# grid Search CV call 
gridCV=GridSearchCV(estimator = LR, param_grid=param, cv=kfolds,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)
gridCV.fit(X_train,np.ravel( y_train))

# random Search CV call

randomCV=RandomizedSearchCV(estimator =LR, param_distributions=param, cv=kfolds,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

randomCV.fit(X_train,y_train)

print("**************Logistic Grid Search CV********************")
print(f"The best {score_measure} score is {gridCV.best_score_}")
print(f"with parameters: {gridCV.best_params_}")

print(f"Best Estimators in grid cv :{gridCV.best_estimator_}")


print("**************Logistic Random Search CV********************")
print(f"The best {score_measure} score is {randomCV.best_score_}")
print(f"with parameters: {randomCV.best_params_}")

print(f"Best Estimators in grid cv :{randomCV.best_estimator_}")



Fitting 5 folds for each of 36 candidates, totalling 180 fits
Fitting 5 folds for each of 10 candidates, totalling 50 fits
**************Logistic Grid Search CV********************
The best precision score is 0.9199999999999999
with parameters: {'C': 0.01, 'penalty': 'l2', 'solver': 'newton-cg'}
Best Estimators in grid cv :LogisticRegression(C=0.01, solver='newton-cg')
**************Logistic Random Search CV********************
The best precision score is 0.9199999999999999
with parameters: {'solver': 'newton-cg', 'penalty': 'l2', 'C': 0.01}
Best Estimators in grid cv :LogisticRegression(C=0.01, solver='newton-cg')
Wall time: 4.55 s


### b. parameters for grid & random search Cv for SVM

In [13]:
%%time
score_measure = "precision"
kfolds = 5

param = {
     'C': [0.1,1, 10, 100], 
     'gamma': [1,0.1,0.001,0.0001],
    'kernel': ['linear','rbf','poly'],
    'degree':[2,3,4],
    
}

# call SVC
SVC=SVC(max_iter=25000)

# grid Search CV call 
gridCV=GridSearchCV(estimator = SVC, param_grid=param, cv=kfolds,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)
gridCV.fit(X_train,np.ravel( y_train))

# random Search CV call
randomCV=RandomizedSearchCV(estimator =SVC, param_distributions=param, cv=kfolds,n_iter=500,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)
randomCV.fit(X_train,y_train)

print("************** SVM Grid Search CV********************")
print(f"The best {score_measure} score is {gridCV.best_score_}")
print(f"with parameters: {gridCV.best_params_}")

print(f"Best Estimators in grid cv :{gridCV.best_estimator_}")


print("**************SVM Random Search CV********************")
print(f"The best {score_measure} score is {randomCV.best_score_}")
print(f"with parameters: {randomCV.best_params_}")

print(f"Best Estimators in grid cv :{randomCV.best_estimator_}")




Fitting 5 folds for each of 144 candidates, totalling 720 fits
Fitting 5 folds for each of 144 candidates, totalling 720 fits
************** SVM Grid Search CV********************
The best precision score is 0.8600000000000001
with parameters: {'C': 0.1, 'degree': 2, 'gamma': 0.1, 'kernel': 'rbf'}
Best Estimators in grid cv :SVC(C=0.1, degree=2, gamma=0.1, max_iter=25000)
**************SVM Random Search CV********************
The best precision score is 0.8600000000000001
with parameters: {'kernel': 'rbf', 'gamma': 0.1, 'degree': 2, 'C': 0.1}
Best Estimators in grid cv :SVC(C=0.1, degree=2, gamma=0.1, max_iter=25000)
Wall time: 1min 17s


### c.  parameters for grid & random search CV for decision tree

In [23]:
%%time

score_measure = "precision"
kfolds = 5

param = {
    'min_samples_split': np.arange(1,15),  
    'min_samples_leaf': np.arange(1,15),
    'min_impurity_decrease': np.arange(0.0001, 0.01),
    'max_leaf_nodes': np.arange(5, 15), 
    'max_depth': np.arange(1,15), 
    'criterion': ['entropy', 'gini'],
}
# Call DT
dt=DecisionTreeClassifier()

# grid Search CV call 
gridCV=GridSearchCV(estimator = dt, param_grid=param, cv=kfolds,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)
gridCV.fit(X_train,np.ravel( y_train))

# random Search CV call
randomCV=RandomizedSearchCV(estimator =dt, param_distributions=param, cv=kfolds, n_iter=500,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)
randomCV.fit(X_train, y_train)

print("************** Decision tree Grid Search CV********************")
print(f"The best {score_measure} score is {gridCV.best_score_}")
print(f"with parameters: {gridCV.best_params_}")

print(f"Best Estimators in grid cv :{gridCV.best_estimator_}")


print("**************Decision tree Random Search CV********************")
print(f"The best {score_measure} score is {randomCV.best_score_}")
print(f"with parameters: {randomCV.best_params_}")

print(f"Best Estimators in grid cv :{randomCV.best_estimator_}")




Fitting 5 folds for each of 54880 candidates, totalling 274400 fits
Fitting 5 folds for each of 500 candidates, totalling 2500 fits
************** Decision tree Grid Search CV********************
The best precision score is 0.728143274853801
with parameters: {'criterion': 'gini', 'max_depth': 8, 'max_leaf_nodes': 13, 'min_impurity_decrease': 0.0001, 'min_samples_leaf': 2, 'min_samples_split': 14}
Best Estimators in grid cv :DecisionTreeClassifier(max_depth=8, max_leaf_nodes=13,
                       min_impurity_decrease=0.0001, min_samples_leaf=2,
                       min_samples_split=14)
**************Decision tree Random Search CV********************
The best precision score is 0.7145871143375679
with parameters: {'min_samples_split': 14, 'min_samples_leaf': 2, 'min_impurity_decrease': 0.0001, 'max_leaf_nodes': 12, 'max_depth': 14, 'criterion': 'gini'}
Best Estimators in grid cv :DecisionTreeClassifier(max_depth=14, max_leaf_nodes=12,
                       min_impurity_decrease

## Analysis
- Universal bank dataset we applied different model with different parame ..
- model used for Universal Bank  Logistic , SVM (different kernels like liner,poly,rbf), Decision tree. 
- mainly we focused on precision because precision can dependes on actual value with predicted value.
- logistic model got precision value 0.9 in both random & grid search with  'C': 0.01, 'penalty': 'l2','solver':'newton-cg'
- SVM model got precision value 0.86 in both random & grid search with C': 0.1, 'degree': 2, 'gamma': 0.1, 'kernel': 'rbf'
- decision tree model best precision score .72 @ 'criterion': 'gini', 'max_depth': 8, 'max_leaf_nodes': 13, 'min_impurity_decrease': 0.0001, 'min_samples_leaf': 2,  'min_samples_split': 14 grid search
- decision tree precision value .71 @ min_samples_split': 14, 'min_samples_leaf': 2, 'min_impurity_decrease': 0.0001, 'max_leaf_nodes': 12, 'max_depth': 14, 'criterion': 'gini' in random seach 
- from decision tree best precision @ grid search
- from all above model logistic has best precision value.
- Based on Logistic model we can deploy and predict. Cilent will get good returns if they used this model.